# WFiApp

Aplikacja będzie wykorzystywać `LazyColumn`, gdzie każdym elementem na liście będzie `Card`. Dodamy również podstawową obsługę gestów oraz wykorzystamy `Compose Navigation` aby otworzyć nowy ekran w którym zaprezentowany będzie bardziej szczegółowy opis wybranego elementu listy. Będzie to wersja WFiApp z poprzedniego modułu, tym razem wykonana w `Compose`.

<table><tr><td><img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExMmI2ZDBhMDQwZTgxZGVmMGE4YTc5NjQ5OTkxMjhjY2Y0NjVmMzFkYiZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/eEqzr1ktXtjwc1yZgb/giphy.gif" width="200" /></td><td><img src="https://media4.giphy.com/media/Acr6sC2y1H1SY2pxEj/giphy.webp" width="200" /></td><td><img src="https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExZTg3MTczYTA5YjNkMGUzODczMjRmNTE3YmNjODhlODRhZWE4MjNiMCZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/fxAJ5nVpI08d1snvpy/giphy.gif" width="350" /></td></tr></table>

Będziemy wykorzystywać `Compose Navigation`, więc dodajmy odpowiednią zależność.

In [ ]:
implementation 'androidx.navigation:navigation-compose:2.5.3'

## Model

Dodajmy model reprezentujący instytut

In [ ]:
data class Institute(
    val title: String,
    val info: String,
    val imageResource: Int // identyfikatory obrazów są przechowywane jako int
)

Następnie dodajmy obiekt `DataProvider` przechowujący listę wszystkich instytutów

In [ ]:
object DataProvider {

    val institutes: ArrayList<Institute> = ArrayList()

    fun getInstituteData(activity: Activity){
        val instituteList = activity.resources.getStringArray(R.array.institute_titles)
        val instituteInfo = activity.resources.getStringArray(R.array.institute_info)
        val instituteImageResources = activity.resources.obtainTypedArray(R.array.institute_images)

        for (i in instituteList.indices) institutes.add(
            Institute(
                instituteList[i],
                instituteInfo[i],
                instituteImageResources.getResourceId(i, 0)
            )
        )

        instituteImageResources.recycle()
    }
}

## Lista - `Card` + `LazyColumn`

<img src="https://i.ibb.co/PgKQ6nS/image.png" width="350" />

Chcemy wyświewtlić listę wszystkich instytutów - każde pola będzie zawierać grafikę, tytuł znajdujący się na grafice (wraz z gradientem, aby poprzwić widoczność tekstu), oraz pole tekstowe zawierające podstawowe informacje. Rozpocznijmy od utworzenia naszej karty, dodajmy funkcję `Composable`, która renderuje kartę z obrazem.

In [ ]:
@Composable
fun ImageCard(
    painter: Painter,
    contentDescription: String,
    title: String,
    info: String,
    modifier: Modifier = Modifier
){

ImageCard przyjmuje kilka parametrów:
- `painter` - obiekt typu `Painter`, który reprezentuje obraz do wyświetlenia.
- `contentDescription` - opis zawartości obrazu, który jest używany w celach dostępności.
- `title` - tytuł karty, który będzie wyświetlany.
- `info` - dodatkowe informacje na temat karty, które będą wyświetlane.
- `modifier` - modyfikator, który kontroluje wygląd i rozmiar karty. Jest opcjonalny i domyślnie ustawiony na `Modifier`.

Wewnątrz funkcji zdefiniujmy `Card`

In [ ]:
Card(
    modifier = modifier
        .fillMaxWidth()
        .padding(15.dp)
        .clickable { },
    shape = RoundedCornerShape(15.dp),
    elevation = CardDefaults.cardElevation(defaultElevation = 10.dp)
) {}

`Card` jest komponentem, który używamy do tworzenia pojemników na inne komponenty, które mogą być stylizowane jako karty. Przyjmuje kilka parametrów, które definiują jego wygląd i zachowanie:
- `modifier` - modyfikator który kontroluje wygląd i rozmiar karty. W tym przypadku, modyfikator składa się z kilku operacji, takich jak `fillMaxWidth()` (wypełnienie karty na szerokość dostępnej przestrzeni), `padding(15.dp)` (dodanie wewnętrznego marginesu 15 jednostek) i `clickable {}` (ustawienie obsługi kliknięcia na kartę).
- `shape` - kształt karty. W tym przypadku, używany jest `RoundedCornerShape(15.dp)`, co oznacza, że karta będzie miała zaokrąglone narożniki o promieniu 15 jednostek.
- `elevation` - podwyższenie karty. W tym przypadku, używany jest `CardDefaults.cardElevation(defaultElevation = 10.dp)`, co oznacza, że karta będzie miała domyślne podwyższenie o wartości 10 jednostek.

Ponieważ sama grafika, kolor pod tekstem oraz tekst znajdują się na stosie (jedne na drugich), wewnątrz `Card` definiujemy `Box` - pozwala dodawać elementy na stos (podobnie jak `Column` dodaje elementuy jeden pod drugim i `Row` jeden obok drugiego, `Box` dodaje elementy jeden na drugi).

In [ ]:
Box(modifier = Modifier
    .height(200.dp)
    .fillMaxWidth()
   ){}

Wysokość `Box` definiujemy na `200.dp` i wypełniamy całą dostępną szerokość. Wewnątrz chcemy umieścić trzy elementy, rozpocznijmy od dodania grafiki. Zdefiniujmy funkcję `Composable` `InstituteImage`

In [ ]:
@Composable
private fun InstituteImage(
    painter: Painter,
    contentDescription: String
) {
    Image(
        painter = painter,
        contentDescription = contentDescription,
        contentScale = ContentScale.Crop,
        modifier = Modifier.fillMaxWidth()
    )
}

`InstituteImage` przyjmuje dwa parametry:
- `painter` - obiekt typu `Painter`, który reprezentuje obraz do wyświetlenia.
- `contentDescription` - opis zawartości obrazu, który jest używany w celach dostępności.
Wewnątrz `InstituteImage` używamy komponentu `Image`, który renderuje obrazek. Przekazujemy `painter` jako parametr do `Image`, który reprezentuje obraz do wyświetlenia, a `contentDescription` jako opis dostępności dla obrazu. Dodatkowo, ustawiamy `contentScale` na `ContentScale.Crop`, co oznacza, że obrazek będzie przycięty, aby zmieścić się w ramach komponentu, jednocześnie zachowując proporcje. Modyfikator `Modifier.fillMaxWidth()` został zastosowany do komponentu `Image`, co oznacza, że obrazek wypełni całą dostępną szerokość.

Drugim elementem będzie gradient koloru, który będzie wyświetlany pod tekstem. Dodajmy funkcję `Composable` `Gradient`

In [ ]:
@Composable
private fun Gradient() {
    Box(
        modifier = Modifier
            .fillMaxSize()
            .background(
                Brush.verticalGradient(
                    colors = listOf(
                        Color.Transparent,
                        Color.Black
                    ),
                    startY = 450f
                )
            )
    )
}

Wewnątrz funkcji `Gradient` używamy komponentu `Box`, który działa jako kontener dla innych komponentów. W modyfikatorze `modifier` komponentu `Box`, zastosowano modyfikator `Modifier.fillMaxSize()`, który sprawia, że `Box` wypełnia całą dostępną przestrzeń. Użyto modyfikatora `.background()`, aby ustawić tło `Box` jako gradient. Wewnątrz modyfikatora `.background()`, użyto `Brush.verticalGradient()`, który tworzy pionowy gradient. W parametrze `colors` metody `Brush.verticalGradient()`, przekazano listę kolorów, w tym `Color.Transparent` i `Color.Black`. Gradient będzie płynnie przechodził od przezroczystego koloru na górze do czarnego koloru na dole. Dodatkowo, użyto parametru `startY = 450f`, który definiuje początek gradientu od 450 pikseli od góry.

Ostatnim elementem jest `Text` wyświetlający tytuł

In [ ]:
Text(
    text = title,
    modifier = Modifier
        .align(Alignment.BottomStart)
        .padding(12.dp),
    fontSize = 24.sp,
    style = TextStyle(color = Color.White)
)

Pod komponentem `Box`, w którym umieszczamy grafikę, gradient oraz tytuł, dodajmy jeszcze jeden komponent wyświetlający `info`

In [ ]:
Box(modifier = Modifier
    .fillMaxWidth()
    .height(45.dp)
) {
    Text(text = info)
}

Pełny kod funkcji `ImageCard`

In [ ]:
@Composable
fun ImageCard(
    painter: Painter,
    contentDescription: String,
    title: String,
    info: String,
    modifier: Modifier = Modifier
){
        Card(
            modifier = modifier
                .fillMaxWidth()
                .padding(15.dp)
                .clickable { },
            shape = RoundedCornerShape(15.dp),
            elevation = CardDefaults.cardElevation(defaultElevation = 10.dp)
        ) {
            Box(modifier = Modifier
                .height(200.dp)
                .fillMaxWidth()){
                InstituteImage(painter, contentDescription)
                Gradient()
                Text(
                    text = title,
                    modifier = Modifier
                        .align(Alignment.BottomStart)
                        .padding(12.dp),
                    fontSize = 24.sp,
                    style = TextStyle(color = Color.White)
                )
            }
            Box(modifier = Modifier
                .fillMaxWidth()
                .height(45.dp)
            ) {
                Text(text = info)
            }
        }
}

Dodajmy funkcję `InstituteList` wyświetlającą wszystkie elementy listy danych z `DataProvider` za pomocą `ImageCard`

In [ ]:
@Composable
fun InstituteList(){
    val data by remember {
        mutableStateOf(DataProvider.institutes.toList())
    }

    LazyColumn(
        modifier = Modifier.fillMaxSize()
    ){
        items(data.size){index ->
            Row(
                modifier = Modifier
                    .fillMaxSize()
                    .padding(4.dp)
                    .clickable { /*TODO*/ },
                horizontalArrangement = Arrangement.SpaceAround,
                verticalAlignment = Alignment.CenterVertically
            ) {
                ImageCard(
                    painter = painterResource(id = data[index].imageResource),
                    contentDescription = data[index].title,
                    title = data[index].title,
                    info = data[index].info
                )
            }
        }
    }
}

`InstituteList` jest funkcją komponującą, która renderuje listę instytutów. Używamy `mutableStateOf` w celu utworzenia zmiennej stanu `data`, która jest zmienną, przechowującą listę instytutów. Wywołanie `DataProvider.institutes.toList()` zwraca listę instytutów z dostawcy danych.

Wewnątrz `LazyColumn`, używamy bloku `items` do iteracji po każdym elemencie `data` i generowania wierszy w liście. Używamy komponentu `Row`, który renderuje wiersz z komponentami ułożonymi w poziomie. W modyfikatorze `Row`, używamy `Modifier.fillMaxSize()`, który sprawia, że Row wypełnia całą dostępną przestrzeń. Dodatkowo, używamy modyfikatora `.padding(4.dp)` do dodania marginesu 4 jednostek wokół wiersza. Używamy modyfikatora `.clickable { /*TODO*/ }`, aby dodać obsługę kliknięcia wiersza. Zrobimy to na późniejszym etapie. Wewnątrz `Row` używamy komponentu `ImageCard` (zdefiniowanego wcześniej), który renderuje kartę z obrazkiem dla każdego instytutu. Przekazujemy odpowiednie parametry do `ImageCard`, takie jak `painter` (obraz do wyświetlenia), `contentDescription` (opis obrazu), title (tytuł instytutu) i `info` (informacje o instytucie).

Fuinkcję `InteitutList` możemy wywołać w `MainActivity`

<img src="https://media4.giphy.com/media/Acr6sC2y1H1SY2pxEj/giphy.webp" width="200" />

## Detail Screen

Przygotujmy widok ekranu dla *szczegółu* (w koncepcji *master-detail*). Wykorzystamy wcześniej zaimplementowaną funkcję `Composable` `ImageCard`, wpierw musimy ją zmodyfikować.

In [ ]:
@Composable
fun ImageCard(
    painter: Painter,
    contentDescription: String,
    title: String,
    info: String,
    index: Int,
    modifier: Modifier = Modifier,
    onClick: (Int) -> Unit = { },
    body: String = ""
) {
    Card(
        modifier = modifier
            .fillMaxWidth()
            .padding(15.dp)
            .clickable { onClick(index) },
        shape = RoundedCornerShape(15.dp),
        elevation = CardDefaults.cardElevation(defaultElevation = 10.dp)
    ) {
        Box(
            modifier = Modifier
                .height(200.dp)
                .fillMaxWidth()
        ) {
            InstituteImage(painter, contentDescription)
            Gradient()
            Text(
                text = title,
                modifier = Modifier
                    .align(Alignment.BottomStart)
                    .padding(12.dp),
                fontSize = 24.sp,
                style = TextStyle(color = Color.White)
            )
        }
        Column(
            modifier = Modifier
                .fillMaxWidth()
        ) {
            Text(text = info)
            Text(text = body, modifier = Modifier.padding(start = 16.dp, top = 24.dp, end = 16.dp ))
        }
    }
}

Funkcje `InstituteImage` oraz `Gradient` pozostają bez zmian. Nasza funkcja będzie przyjmować kilka dodatkowych parametrów.

In [ ]:
fun ImageCard(
    painter: Painter,
    contentDescription: String,
    title: String,
    info: String,
    index: Int,
    modifier: Modifier = Modifier,
    onClick: (Int) -> Unit = { },
    body: String = ""
)

Przekazujemy funckcję `onClick`, którą wywołujemy przy kliknięciuu na element, a którą zaimplementujemy w komponencie odpowiedzialnym za nawigację. Ponieważ będziemy wykorzystywać tą funkcję w `Composable` odpowiedzialnych za renderowanie obu ekranów, a kliknięcie chcemy obsłużyć tylko na liście, dodajemy domyślną implementację w postaci pustej funkcji (`onClick: (Int) -> Unit = { }`). Kolejnym dodatkowym elementem jest `body`, jest to tekst który chcemy wyświetlić na ekranie `Detail`, na liście tekst pozostaje pusty, więc również stosuję domyślną wartość (``body: String = ""`). Ostanią zmianą w argumentach jest przekazanie indeksu (`index`) elementu - musimy przekazać indeks aby wywołać `onClick` z odpowiednim argumentem.

Kolejną zmianą jest dodanie `clickable` do modyfikatora komponentu `Card`, oraz wywołanie w nim metody `onClick`.

In [ ]:
...
Card(
    modifier = modifier
        .fillMaxWidth()
        .padding(15.dp)
        .clickable { onClick(index) }, // nasza karta będzie "klikalna"
...

Ostatnią modyfikacją jest dodanie jednego pola `Text` w statunku do poprzedniej wersji, ponieważ chcemy to pole ustawić pod poprzednim, całość umieszczamy w komponencie `Column`

In [ ]:
Column(
    modifier = Modifier
        .fillMaxWidth()
) {
    Text(text = info)
    Text(text = body, modifier = Modifier.padding(start = 16.dp, top = 24.dp, end = 16.dp )) // tutaj wyświetlimy tekst "ciała"
}

Sam tekst dodaję, dla uproszczenia, jako stałą

In [ ]:
val lorem = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. " +
    "Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in " +
    "reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in " +
    "culpa qui officia deserunt mollit anim id est laborum"

Powróćmy do komponentu `InstituteList`, teraz funkcja będzie przyjmować funkcję `onClick` jako argument, który przekażemy do funkcji `ImageCard`

In [ ]:
@Composable
fun InstituteList(onClick: (Int) -> Unit) {
...
    ImageCard(
        painter = painterResource(id = data[index].imageResource),
        contentDescription = data[index].title,
        title = data[index].title,
        info = data[index].info,
        onClick = onClick, // przekazujemy funkcję onClick
        index = index
    )
...
}

Przejdźmy do ekranu `Detail`

In [ ]:
@Composable
fun DetailView(
    id: String?
){
    val data by remember {
        mutableStateOf(DataProvider.institutes[id!!.toInt()])
    }

    Box(modifier = Modifier.fillMaxSize()) {
        ImageCard(
            painter = painterResource(id = data.imageResource),
            contentDescription = data.title,
            title = data.title,
            info = data.info,
            index = id?.toInt() ?: 0,
            modifier = Modifier
                .fillMaxSize()
                .padding(0.dp),
            body = lorem
        )
    }

Będzie on przyjmował `id` - czyli indeks klikniętego elementu. Na podstawie tego indeksu, pobieramy dane z `DataProvider` i wyświetlamy na ekranie. W wywłaniu funkcji `ImageCard` pomijam `onClick` - tutaj nie chcę go implementować, więc pozostawiam domyślnie pustą funkcję, przekazuję `body` wcześniej dodany tekst w stałej `lorem` i tym razem nie korzystam z domyślnego modyfikatora. `id` zostanie przekazany jako `String?` (wymóg `Compose Navigation`), ponieważ ta wartość jest typu `String?`, w przypadku gdy będzie miał wartość `null` funkkcję `ImageCard` wywołamy z `index = 0` (`index = id?.toInt() ?: 0`).

## Navigation

Ponieważ będziemy przekazywać argument w postaci `id` między ekranami, aby ułatwić nieco ten proces, dodajmy metodę `withArguments` do klasy `Screens`

In [ ]:
sealed class Screens(val route: String) {
    object ListScreen : Screens("list")
    object DetailScreen : Screens("detail")

    fun withArgs(vararg args: String): String{
        return buildString {
            append(route)
            args.forEach { arg ->
                append("/$arg")
            }
        }
    }
}

Teraz chcąc przekazać argumenty możemy wywołać funkcję, która utworzy odpowiedni `String` i przekaże go do nawigacji.

Utwórzmy komponent `Navigation` (który wywołamy w `MainActivity`). Przykładowo:

- chcąc przekazać jeden argument `id`: `withArguments(id)` **->** `detail/id`
- chcąc przekazać cztery argumenty `id`, `name`, `state`, `num` **->** `detail/id/name/state/num`

Kolejnym elementem będzie utworzenie komponentu `Navigation`, który wywołamy w `MainActivity`

In [ ]:
@Composable
fun Navigation() {
    val navController = rememberNavController()

    NavHost(navController = navController, startDestination = Screens.ListScreen.route) {

        composable(route = Screens.ListScreen.route ) {
            InstituteList{navController.navigate(Screens.DetailScreen.withArgs(it.toString()))}
        }

        composable(
            route = Screens.DetailScreen.route + "/{index}",
            arguments = listOf(navArgument("index") {
            type = NavType.StringType
        })) {
            DetailView(id = it.arguments?.getString("index"))
        }
    }
}

Jak w poprzednich przykładach, dodajemy `navController` i tworzymy `NavHost`. Jako punkt startowy wskazujemy listę i tworzymy dwa `composable` po których chcemy nawigować. Pierwszym jest lista wszystkich elementów, funkcja `InstituteList` przyjmuje jako argument funkcję `onClick`, którą wywołujemy po kliknięciu w dowolny element listy - w wyniku czego chcemy przejść do ekranu `Detail`. Jaki argument chcę przekazać `id` - jeżeli użytkownik kliknie trzeci element, chcę przekazać `id` tego elementu, aby na ekranie `Detail` móc wyciągnąć dane z `DataProvider` o odpowiednim indeksie z list `institutes`. Wywołuję funkcję `withArgs` i przekazuję `id`

In [ ]:
composable(route = Screens.ListScreen.route ) {
    InstituteList{navController.navigate(Screens.DetailScreen.withArgs(it.toString()))}
}

Mopikm drugim elementeem `composable` jest `DetailView`, tutaj musimy odebrać argument i przekazać go jako parametr funkcji `DetailView`

In [ ]:
composable(
    route = Screens.DetailScreen.route + "/{index}",
    arguments = listOf(navArgument("index") {
    type = NavType.StringType // zazwyczaj jawnie podaje się typ argumentu
})) {
    DetailView(id = it.arguments?.getString("index"))
}

Możemy przetestować aplikację

<img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExMmI2ZDBhMDQwZTgxZGVmMGE4YTc5NjQ5OTkxMjhjY2Y0NjVmMzFkYiZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/eEqzr1ktXtjwc1yZgb/giphy.gif" width="200" />

## LazyGrid

Ostatnim elementem aplikacji będzie zmiana ilości kolumn wyświetlanych w przypadku zmiany orientacji urządzenia. Komponent `LazyColumn` wyświetla tylko jedną kolumnę, jeżeli chcemy mieć możliwość wyświetlania większej ilości musimy użyć komponentu `LazyGrid` - w tym przypadku `LazyVerticalGrid`

In [ ]:
@Composable
fun InstituteList(onClick: (Int) -> Unit) {
    val data by remember { mutableStateOf(DataProvider.institutes.toList()) }
    var numOfColumns by remember { mutableStateOf(1) }

    numOfColumns = getColumnsFromOrientation()

    LazyVerticalGrid(
        columns = GridCells.Fixed(numOfColumns),
        modifier = Modifier.fillMaxSize()
    ) {...}

Do funkcji `InstituteList` dodajemy pole `numOfColumns`, którego wartość będziemy modyfikować zależnie od orientacji - inicjujemy wartością 1. Ilość kolumn ustalimy w funkcji `getColumnsFromOrientation`, następnie przekażemy ilość do `LazyVerticalGrid`

In [ ]:
columns = GridCells.Fixed(numOfColumns)

`columns` jest zmienną, która będzie reprezentować liczbę kolumn w siatce (`grid`). `GridCells.Fixed(numOfColumns)` to konstruktor obiektu `GridCells.Fixed`, gdzie `numOfColumns` jest liczbą określającą ilość kolumn w siatce. Dzięki temu ustawieniu, siatka będzie miała stałą liczbę kolumn.

In [ ]:
@Composable
private fun getColumnsFromOrientation(): Int {
    var orientation by remember { mutableStateOf(Configuration.ORIENTATION_PORTRAIT) }
    val configuration = LocalConfiguration.current

    LaunchedEffect(configuration) {
        snapshotFlow { configuration.orientation }
            .collect { orientation = it }
    }

    return when (orientation) {
        Configuration.ORIENTATION_PORTRAIT -> { 1 }
        else -> { 2 }
    }
}

Wewnątrz funkcji definiujemy zmienną `orientation`, która jest zmienną stanu (`mutableStateOf`) przechowującą aktualną orientację urządzenia. Początkowo ustawiamy ją na `Configuration.ORIENTATION_PORTRAIT`. Wykorzystujemy `LocalConfiguration.current`, aby uzyskać dostęp do bieżącej konfiguracji urządzenia. Używamy `LaunchedEffect` do obsługi zmiany konfiguracji urządzenia. W bloku `LaunchedEffect`, używamy `snapshotFlow` w celu utworzenia strumienia z obecną wartością `configuration.orientation` (strumienie omówimy dokładnie w jednym w kolejnych modułów), a następnie zbieramy (`collect`) wartości strumienia i przypisujemy je do zmiennej `orientation`. Zwracamy wartość liczby kolumn w zależności od orientacji urządzenia. Jeśli `orientation` jest równe `Configuration.ORIENTATION_PORTRAIT`, zwracamy 1, w przeciwnym razie zwracamy 2.

Możemy przetestować aplikację.

<img src="https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExZTg3MTczYTA5YjNkMGUzODczMjRmNTE3YmNjODhlODRhZWE4MjNiMCZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/fxAJ5nVpI08d1snvpy/giphy.gif" width="350" />